# PyTorch実装

## プロジェクト１：車の燃費を予測する

In [1]:
import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
columns = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration', 'Model Year', 'Origin']
df = pd.read_csv(url, names=columns, na_values='?', comment='\t', sep=' ', skipinitialspace=True)

# 不要な行を削除
df = df.dropna()
df = df.reset_index(drop=True)
# 訓練データセットとテストデータに分割
import sklearn
df_train, df_test = sklearn.model_selection.train_test_split(df, train_size=0.8, random_state=1)
train_stats = df_train.describe().transpose()
numeric_column_names = ['Cylinders', 'Displacement', 'Horsepower', 'Weight', 'Acceleration']
df_train_norm, df_test_norm = df_train.copy(), df_test.copy()
for col_name in numeric_column_names:
    mean = train_stats.loc[col_name, 'mean']
    std = train_stats.loc[col_name, 'std']
    # 先に float64 に変換しておく（警告対策）
    df_train_norm[col_name] = df_train_norm[col_name].astype('float64')
    df_test_norm[col_name] = df_test_norm[col_name].astype('float64')
    # z-score 標準化
    df_train_norm[col_name] = (df_train_norm[col_name] - mean) / std
    df_test_norm[col_name] = (df_test_norm[col_name] - mean) / std

df_train_norm.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
203,28.0,-0.824303,-0.901020,-0.736562,-0.950031,0.255202,76,3
255,19.4,0.351127,0.413800,-0.340982,0.293190,0.548737,78,1
72,13.0,1.526556,1.144256,0.713897,1.339617,-0.625403,72,1
235,30.5,-0.824303,-0.891280,-1.053025,-1.072585,0.475353,77,1
37,14.0,1.526556,1.563051,1.636916,1.470420,-1.359240,71,1


In [2]:
# Model Yearをバケットに変換 例：{bucket0: 1970, bucket1: 1971, ..., bucket7: 1977}
import torch
boundaries = torch.tensor([73, 76, 79])
v = torch.tensor(df_train_norm['Model Year'].values)
df_train_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)
v = torch.tensor(df_test_norm['Model Year'].values)
df_test_norm['Model Year Bucketed'] = torch.bucketize(v, boundaries, right=True)
numeric_column_names.append('Model Year Bucketed')

In [3]:
# one-hotエンコーディング
from torch.nn.functional import one_hot
total_origin = len(set(df_train_norm['Origin']))
origin_encoded = one_hot(torch.from_numpy(df_train_norm['Origin'].values) % total_origin)
x_train_numeric = torch.tensor(df_train_norm[numeric_column_names].values)
x_train = torch.cat([x_train_numeric, origin_encoded], 1).float()
origin_encoded = one_hot(torch.from_numpy(df_test_norm['Origin'].values) % total_origin)
x_test_numeric = torch.tensor(df_test_norm[numeric_column_names].values)
x_test = torch.cat([x_test_numeric, origin_encoded], 1).float()

In [4]:
y_train = torch.tensor(df_train_norm['MPG'].values).float()
y_test = torch.tensor(df_test_norm['MPG'].values).float()

In [5]:
from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(x_train, y_train)
batch_size = 8
torch.manual_seed(1)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [6]:
import torch.nn as nn
hidden_units = [8, 4]
input_size = x_train.shape[1]
all_layers = []
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 1))
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Linear(in_features=9, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=4, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4, out_features=1, bias=True)
)

In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
torch.manual_seed(1)
num_epochs = 200
log_epochs = 20
for epoch in range(num_epochs):
    loss_hist_train = 0.0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)[:, 0] # すべての行（:）から、0番目の列（0）だけを取り出す
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist_train += loss.item()
    if epoch % log_epochs == 0:
        print(f'Epoch {epoch}, Loss: {loss_hist_train / len(train_dl)}')


Epoch 0, Loss: 627.325813293457
Epoch 20, Loss: 12.1157776594162
Epoch 40, Loss: 8.858593660593034
Epoch 60, Loss: 7.987342631816864
Epoch 80, Loss: 7.664143167436123
Epoch 100, Loss: 7.636634939908982
Epoch 120, Loss: 7.644381168484688
Epoch 140, Loss: 7.975517475605011
Epoch 160, Loss: 7.91185485124588
Epoch 180, Loss: 7.8687692195177075


In [9]:
with torch.no_grad():
    pred = model(x_test.float())[:, 0]
    loss = loss_fn(pred, y_test)
    print(f'Test Loss: {loss.item():.4f}')
    print(f'Test MAE: {nn.L1Loss()(pred, y_test).item():.4f}')

Test Loss: 9.7880
Test MAE: 2.0775


## プロジェクト２：MNISTの手書きの数字を分類する

In [10]:
import torchvision
from torchvision import transforms
image_path = './'
transform = transforms.Compose([transforms.ToTensor()]) # [0, 255] -> [0.0, 1.0] に変換
mnist_train_dataset = torchvision.datasets.MNIST(image_path, train=True, download=False, transform=transform)
mnist_test_dataset = torchvision.datasets.MNIST(image_path, train=False, download=False, transform=transform)
batch_size = 64
torch.manual_seed(1)
train_dl = DataLoader(mnist_train_dataset, batch_size=batch_size, shuffle=True)

In [11]:
hidden_units = [32, 16]
image_size = mnist_train_dataset[0][0].shape
input_size = image_size[0] * image_size[1] * image_size[2]
all_layers = [nn.Flatten()]
for hidden_unit in hidden_units:
    layer = nn.Linear(input_size, hidden_unit)
    all_layers.append(layer)
    all_layers.append(nn.ReLU())
    input_size = hidden_unit

all_layers.append(nn.Linear(hidden_units[-1], 10))
all_layers.append(nn.Softmax(dim=1))  # 出力を確率に変換
model = nn.Sequential(*all_layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=32, bias=True)
  (2): ReLU()
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Linear(in_features=16, out_features=10, bias=True)
  (6): Softmax(dim=1)
)

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
torch.manual_seed(1)
num_epochs = 20
for epoch in range(num_epochs):
    accuracy_hist_train = 0.0
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        is_correct = (pred.argmax(dim=1) == y_batch).float()
        accuracy_hist_train += is_correct.sum()

    accuracy_hist_train /= len(train_dl.dataset)
    print(f'Epoch {epoch}, Accuracy: {accuracy_hist_train:.4f}')

Epoch 0, Accuracy: 0.7754
Epoch 1, Accuracy: 0.9152
Epoch 2, Accuracy: 0.9299
Epoch 3, Accuracy: 0.9384
Epoch 4, Accuracy: 0.9433
Epoch 5, Accuracy: 0.9470
Epoch 6, Accuracy: 0.9508
Epoch 7, Accuracy: 0.9538
Epoch 8, Accuracy: 0.9564
Epoch 9, Accuracy: 0.9575
Epoch 10, Accuracy: 0.9596
Epoch 11, Accuracy: 0.9613
Epoch 12, Accuracy: 0.9626
Epoch 13, Accuracy: 0.9642
Epoch 14, Accuracy: 0.9651
Epoch 15, Accuracy: 0.9674
Epoch 16, Accuracy: 0.9676
Epoch 17, Accuracy: 0.9688
Epoch 18, Accuracy: 0.9699
Epoch 19, Accuracy: 0.9710


In [14]:
pred = model(mnist_test_dataset.data / 255.0)
is_correct = (pred.argmax(dim=1) == mnist_test_dataset.targets).float()
print(f'Test Accuracy: {is_correct.mean():.4f}')

Test Accuracy: 0.9570
